<a href="https://colab.research.google.com/github/luqmansen/notebook-collection/blob/master/kNN_cat_dogs_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tugas Mata Kuliah Data Mining**
## Klasifikasi Data Citra menggunakan KNN

1. Luqman Setyo / Nugroho 21120117120008

2. Heidy novendra / 21120117130035

3. Faiz Noerdiyan / 21120117130060

4. Chandra Purnama / 21120117130070

Sumber dataset

Dogs vs. Cats

https://www.kaggle.com/c/dogs-vs-cats/data

In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
%cd '/content/drive/My Drive/Colab Notebooks/knn-cat-dogs-classifier'

/content/drive/My Drive/Colab Notebooks/knn-cat-dogs-classifier


In [4]:
!wget 'https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/3362/31148/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1586705610&Signature=rXK24eedObdcRI%2BurzUdWbX7u5AKMrbWaQF6brzetZSXxj98s9tHTIdu3ZZ%2BYsFIemWh9%2BpZlyuMvIGcqbVhIeigN0YC9ymXDpAJ4%2BQYpnlKN%2FG5F48hXLDFdrn12k81W%2BkWd6%2FTOrc%2FuER1j%2FxxegO1m61Xs4gnZFNB3KmApquqnJD5qQLCsO%2BrXStOCjP1zc1F7sc1yVLveTCFO7rVIlIAy1mBTVEk8G%2BaqGkWq8OIsF2gr8hbMUfBOg0ZgMtgddI3VfFER0GqoTpEief2QKv788ZC7myqq29kKDu5yNUa68q6DcvEq5pZnXm88PlmNYp7RM2VnF2dIJ4l0gY6Xg%3D%3D&response-content-disposition=attachment%3B+filename%3Ddogs-vs-cats.zip'

The name is too long, 522 chars total.
Trying to shorten...
New name is archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1586705610&Signature=rXK24eedObdcRI+urzUdWbX7u5AKMrbWaQF6brzetZSXxj98s9tHTIdu3ZZ+YsFIemWh9+pZlyuMvIGcqbVhIeigN0YC9ymXDpAJ4+QYpnlKN%2FG5F48hXLDFdrn12k81W+k.
--2020-04-09 15:35:42--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/3362/31148/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1586705610&Signature=rXK24eedObdcRI%2BurzUdWbX7u5AKMrbWaQF6brzetZSXxj98s9tHTIdu3ZZ%2BYsFIemWh9%2BpZlyuMvIGcqbVhIeigN0YC9ymXDpAJ4%2BQYpnlKN%2FG5F48hXLDFdrn12k81W%2BkWd6%2FTOrc%2FuER1j%2FxxegO1m61Xs4gnZFNB3KmApquqnJD5qQLCsO%2BrXStOCjP1zc1F7sc1yVLveTCFO7rVIlIAy1mBTVEk8G%2BaqGkWq8OIsF2gr8hbMUfBOg0ZgMtgddI3VfFER0GqoTpEief2QKv788ZC7myqq29kKDu5yNUa68q6DcvEq5pZnXm88PlmNYp7RM2VnF2dIJ4l0gY6Xg%3D%3D&response-content-disposition=attachment%3B+filename%3Ddogs-vs-cats.zip
Resolving storage.googleapis.com (st

In [0]:
!unzip -q archive*

In [10]:
!unzip -q test1.zip

replace test1/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [0]:
!unzip -q train.zip

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import os
import time

In [0]:
def image_to_feature_vector(image, size=(32, 32)):
	# Konversi gambar menjadi berukuran 32x32 
    
  # return berupa list of number 32x32x3 (3 = channel rgb) = list of 3.072 
	return cv2.resize(image, size).flatten()

In [0]:
def extract_color_histogram(image, bins=(8, 8, 8)):
	# konversi gambar ke hsv
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  # menghitung histogram gambar
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,[0, 180, 0, 256, 0, 256])
	
  # Normalisasi histogram
	if imutils.is_cv2():
		hist = cv2.normalize(hist)
	else:
		cv2.normalize(hist, hist)
  
	# return berupa feature vector 512-d (8 untuk tiap H,S,V sehingga 8x8x8)
	return hist.flatten()

In [0]:
imagePaths = list(paths.list_images("train/"))
# initialize the raw pixel intensities matrix, the features matrix, and labels list
rawImages = []
features = []
labels = []

In [28]:
# loop over the input images
for (i, imagePath) in enumerate(imagePaths):
  # load the image and extract the class label (assuming that our
  # path as the format: /path/to/dataset/{class}.{image_num}.jpg
  image = cv2.imread(imagePath)
  label = imagePath.split(os.path.sep)[-1].split(".")[0]
  # extract raw pixel intensity "features", followed by a color
  # histogram to characterize the color distribution of the pixels in the image
  pixels = image_to_feature_vector(image)
  hist = extract_color_histogram(image)
  # update the raw images, features, and labels matricies, respectively
  rawImages.append(pixels)
  features.append(hist)
  labels.append(label)
	# show an update every 1,000 images
  if i > 0 and i % 1000 == 0:
    print("[INFO] processed {}/{}".format(i, len(imagePaths)))

[INFO] processed 1000/25000
[INFO] processed 2000/25000
[INFO] processed 3000/25000
[INFO] processed 4000/25000
[INFO] processed 5000/25000
[INFO] processed 6000/25000
[INFO] processed 7000/25000
[INFO] processed 8000/25000
[INFO] processed 9000/25000
[INFO] processed 10000/25000
[INFO] processed 11000/25000
[INFO] processed 12000/25000
[INFO] processed 13000/25000
[INFO] processed 14000/25000
[INFO] processed 15000/25000
[INFO] processed 16000/25000
[INFO] processed 17000/25000
[INFO] processed 18000/25000
[INFO] processed 19000/25000
[INFO] processed 20000/25000
[INFO] processed 21000/25000
[INFO] processed 22000/25000
[INFO] processed 23000/25000
[INFO] processed 24000/25000


In [40]:
# show some information on the memory consumed by the raw images
# matrix and features matrix
rawImages = np.array(rawImages)
features = np.array(features)
labels = np.array(labels)
print("[INFO] pixels matrix: {:.2f}MB".format(rawImages.nbytes / (1024 * 1000.0)))
print("[INFO] features matrix: {:.2f}MB".format(features.nbytes / (1024 * 1000.0)))

[INFO] pixels matrix: 75.00MB
[INFO] features matrix: 50.00MB


In [44]:
# partition the data into training and testing splits, using 75%
# of the data for training and the remaining 25% for testing
print("[INFO] constructing training/testing split...")
(trainData, testData, trainLabels, testLabels) = train_test_split(
	features, labels, test_size=0.25, random_state=42)

[INFO] constructing training/testing split...


In [45]:
# partition the data into training and testing splits, using 75%
# of the data for training and the remaining 25% for testing
print("[INFO] constructing training/testing split...")
(trainDataRI, testDataRI, trainLabelsRI, testLabelsRI) = train_test_split(
	rawImages, labels, test_size=0.25, random_state=42)

[INFO] constructing training/testing split...


In [46]:
# train and evaluate a k-NN classifer on the raw pixel intensities
print("[INFO] evaluating raw pixel accuracy...")
model = KNeighborsClassifier(n_neighbors=1,	n_jobs=-1)
model.fit(trainDataRI, trainLabelsRI)
acc = model.score(testDataRI, testLabelsRI)
print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))

[INFO] evaluating raw pixel accuracy...
[INFO] raw pixel accuracy: 54.56%


In [47]:
# train and evaluate a k-NN classifer on the histogram
print("[INFO] evaluating feature pixel accuracy...")
model = KNeighborsClassifier(n_neighbors=1,	n_jobs=-1)
model.fit(trainData, trainLabels)
acc = model.score(testData, testLabels)
print("[INFO] histogram pixel accuracy: {:.2f}%".format(acc * 100))

[INFO] evaluating feature pixel accuracy...
[INFO] histogram pixel accuracy: 57.42%


## Hyperparameter tunning

n_neghbors : jumlah NN k (dari 0 hingga 29, exclusive)

metrics : distance function metrics

In [0]:
params = {"n_neighbors": np.arange(1, 31, 2),
	"metric": ["euclidean", "cityblock"]}

In [0]:
model = KNeighborsClassifier(n_jobs=4)

In [0]:
# Grid Search tuning
print("[INFO] tuning hyperparameters via grid search")
grid = GridSearchCV(model, params, verbose=1)
start = time.time()
grid.fit(trainData, trainLabels)
# evaluate the best grid searched model on the testing data
print("[INFO] grid search took {:.2f} seconds".format(time.time() - start))
acc = grid.score(testData, testLabels)
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(grid.best_params_))

In [48]:
# tune the hyperparameters via a randomized search
print("[INFO] tuning hyperparameters via randomized search")
grid = RandomizedSearchCV(model, params)
start = time.time()
grid.fit(trainData, trainLabels)
# evaluate the best randomized searched model on the testing data
print("[INFO] randomized search took {:.2f} seconds".format(time.time() - start))
acc = grid.score(testData, testLabels)
print("[INFO] randomized search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] randomized search best parameters: {}".format(grid.best_params_))

[INFO] tuning hyperparameters via randomized search
[INFO] randomized search took 2225.99 seconds
[INFO] randomized search accuracy: 64.40%
[INFO] randomized search best parameters: {'n_neighbors': 29, 'metric': 'cityblock'}
